In [3]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [4]:
# make page
paper_size = '11x14 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [5]:
def poly_to_edges(poly):
    bcoords = poly.boundary.coords
    n_edges = len(bcoords)-1
    edges = [LineString((bcoords[i], bcoords[i+1])) for i in range(n_edges)]
    return edges

In [6]:
def inscribe_poly(edges, interp_distances):
    pts = [edge.interpolate(d, normalized=True) for edge,d in zip(edges, interp_distances)]
    return Polygon(pts)

In [7]:
def subdivide_poly(poly, interp_distances):
    edges = poly_to_edges(poly)
    inscribed_poly = inscribe_poly(edges, interp_distances)
    subdivided_poly = gp.merge_Polygons([poly.difference(inscribed_poly), inscribed_poly]).buffer(-0.001)
    return [p for p in subdivided_poly]

In [8]:
def subdivide_gdf_row(parent_row, distances):
    current_generation = parent_row['generation'] + 1
    poly = parent_row['geometry']
    new_polys = subdivide_poly(poly, distances)
    ids =[id(p) for p in new_polys]
    ndf = geopandas.GeoDataFrame({
        'geometry':new_polys,
        'id':ids,
    })
    ndf['distances'] = [distances] * len(ndf)
    ndf['generation'] = current_generation
    ndf['subdivided'] = False
    ndf['parent'] = parent_row['id']
    return ndf

In [9]:
def subdivide_rows(df, index, distances):
    ndfs = []
    for i in index:
        parent_row = df.iloc[i, :]
        ndf = subdivide_gdf_row(parent_row, distances)
        ndfs.append(ndf)
        df.loc[i, 'subdivided'] = True
        merged = df.append(pd.concat(ndfs)).reset_index(drop=True)
    return geopandas.GeoDataFrame(merged)

In [8]:
drawbox.bounds

(20.0, 20.0, 259.4, 335.59999999999997)

In [183]:
# initialize
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.25,
    rad_seq_end=0.22, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
tris = so.triangulate(vertices)
gp.merge_Polygons(tris)


# # initialize
lower_left = (20.0, 20.0)
top = (130, 335)
lower_right = (260, 20)
tris = [Polygon([lower_left, top, lower_right]),]

In [184]:

ids =[id(tri) for tri in tris]
parents_df = geopandas.GeoDataFrame({
    'geometry': tris,
    'id': ids,
    'distances': [[None]] * len(tris),
    'generation': [0] * len(tris),
    'subdivided': [False] * len(tris),
    'parent': [None] * len(tris),
    },
)

In [186]:
child_dfs = []
for _, _df in parents_df.iterrows():
    df = _df.to_frame().T
    distances = [0.7, 0.7, 0.7]
    n_generations = 3
    for generation in range(n_generations):
        index = np.flatnonzero(df['generation'] == generation)
        df = subdivide_rows(df, index, distances)
        
    child_dfs.append(df)
    
all_df = pd.concat(child_dfs)
all_df = geopandas.GeoDataFrame(all_df)

In [187]:
bottom = all_df.query('subdivided == False')

In [ ]:
layers = []
for gen, group in bottom.groupby('generation'):
    layer = gp.merge_Polygons([ls for ls in group.geometry if ls is not None]).buffer(-0.5, cap_style=2, join_style=2).boundary      
    layers.append(layer)

In [55]:
layers = []
for gen, group in bottom.groupby('generation'):
    layer = gp.merge_LineStrings([ls for ls in group.boundary if ls is not None])
    layers.append(layer)

In [189]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-08T01:31:25.656316

In [48]:
sk.vpype(f'linemerge --tolerance 0.1mm linesort')

In [49]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/163_triangle_subdivision.svg')

# division

In [103]:
# make page
paper_size = '17x11 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [104]:
states = geopandas.read_file('/mnt/c/data/side/maps/cb_2018_us_state_20m/cb_2018_us_state_20m.shp')

continental = ~states['NAME'].isin(['Hawaii', 'Alaska', 'Puerto Rico'])

cnt = states.loc[continental,:].unary_union.centroid


In [105]:

states['geometry'] = states['geometry'].scale(xfact=1.3, yfact=-1.5, origin=cnt)
us_poly = gp.make_like(states.loc[continental,:].unary_union, drawbox)

In [106]:
start_poly = us_poly.buffer(50)

In [107]:
border_pts = [start_poly.boundary.interpolate(i, normalized=True) for i in np.linspace(0, 1, 10)]

In [108]:
trcp = gp.RecursiveCirclePacker(start_poly, rad_seq_start=0.18,
    rad_seq_end=0.18, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles] + border_pts)
tris = so.triangulate(vertices)

In [117]:
def choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters, n_per_iter=None):
    if valid.sum() > 0:
        p = df[valid].area / df[valid].area.sum()
        if n_per_iter==None:
            n_per_iter = len(df[valid])
        id_choices = np.random.choice(df.loc[valid, 'id'], size=n_per_iter, p=p, replace=False)
        index = np.flatnonzero(df['id'].isin(id_choices))
        
        
        if n_iters == 0:
            return df
        elif n_iters > 0:
            df = subdivide_rows(df, index, distances)
            n_iters -= 1
            valid = (df['generation'] < max_generation) & (df['parent'].isin(id_choices))
            return choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters)
    else:
        return df

In [152]:

ids =[id(tri) for tri in tris]
df = geopandas.GeoDataFrame({
    'geometry': tris,
    'id': ids,
    'distances': [[None]] * len(tris),
    'generation': [0] * len(tris),
    'subdivided': [False] * len(tris),
    'parent': [None] * len(tris),
    },
)

In [162]:
max_generation = 7
n_subdivisions = 200
distances = [0.3, 0.7, 0.3]

In [154]:
valid = (df['generation'] < max_generation) & ~df['subdivided']
df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=1, n_per_iter=None)

In [163]:
for i in range(n_subdivisions):
    valid = (df['generation'] < max_generation) & ~df['subdivided']
    df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=np.random.randint(1,4), n_per_iter=1)

In [164]:
# valid = (df['generation'] < max_generation) & ~df['subdivided']
# df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=3, n_per_iter=2)

layers = []
bottom = df.query('subdivided == False')
for gen, group in bottom.groupby('generation'):
    layer = gp.merge_LineStrings([ls for ls in group.boundary if ls is not None])
    layers.append(layer)

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-07T00:10:38.621187 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [ ]:
for i in df['id']:
    # top level
    valid = (df['generation'] < max_generation) & ~df['subdivided']
    df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=1, id_choice=i)
        
    

In [ ]:
valid = (df['generation'] < max_generation) & ~df['subdivided']
df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=3)

In [76]:
layers = []
bottom = df.query('subdivided == False')
for gen, group in bottom.groupby('generation'):
    layer = gp.merge_LineStrings([ls for ls in group.boundary if ls is not None])
    diff_layer = layer.intersection(us_poly)
    layers.append(diff_layer)

In [77]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-06T23:52:41.285579

In [ ]:
tolerance=0.1
sk.vpype(f'splitall linemerge --tolerance {tolerance}mm linemerge --tolerance {tolerance}mm linesort')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/162_subdivision.svg')

## circle

In [57]:
class PolygonGrowth(object):
    
    def __init__(self, polygon):
        self.polygons = [polygon]
        self.growth = polygon
        self.growth = Polygon(self.growth.exterior)
        self.obj_func = None
        
    def _repr_svg_(self):
        return self.growth._repr_svg_()
        
    @property
    def lines(self):
        lss = []
        for p in self.polygons:
            b = p.boundary
            if b.type == 'LineString':
                lss.append(b)
            elif b.type == 'MultiLineString':
                lss += list(b)
        return MultiLineString(lss)
        
    def calc_polygon_diff(self, polygon):
        return polygon.difference(self.growth)
        
    def add_polygon_diff(self, polygon):
        polygon_diff = self.calc_polygon_diff(polygon)
        self.polygons.append(polygon_diff)
        self.growth = so.unary_union([self.growth, polygon_diff])
        self.growth = Polygon(self.growth.boundary)
        
    def get_boundary_point(self, boundary_loc=None):
        if boundary_loc is None:
            boundary_loc = np.random.rand()
        return self.growth.boundary.interpolate(boundary_loc, normalized=True)
    
    def get_random_boundary_points(self, n):
        return MultiPoint([self.get_boundary_point() for i in range(n)])
    
    def get_boundary_points(self, boundary_locs):
        return MultiPoint([self.get_boundary_point(b) for b in boundary_locs])
    
    def grow_random_circle_diff(self, rad):
        self.add_polygon_diff(self.get_boundary_point().buffer(rad))
        
    def buffer_growth(self, d):
        self.growth = self.growth.buffer(d)
        self.growth = Polygon(self.growth.exterior)
        
    def filter_points_by_distance(self, points, target_point, descending=True, n_filtered_points=1):
        dists = [target_point.distance(p) for p in points]
        if descending:
            ordered = np.argsort(dists)
        else:
            ordered = np.argsort(-dists)
        return MultiPoint([points[i] for i in ordered[:n_filtered_points]])
    
    def get_LineFactory(self):
        lf = LineFactory()
        for line in self.lines:
            lf.add_line(np.asarray(line.xy))
        return lf
    
    
class TargetPointGrowth(PolygonGrowth):
    
    def grow_circle_diff(self, target_point, rad, n_checks, **kwargs):
        points = self.get_random_boundary_points(n=n_checks)
        fps = self.filter_points_by_distance(points, target_point, **kwargs)
        for fp in fps:
            self.add_polygon_diff(fp.buffer(rad))
            
            

In [151]:
target_point = drawbox.centroid
max_rads = [40, ]
min_rads = [1,]
circles_per_wave = [100, ]
check_ns_per_wave = [ 250, ]
tps = [target_point ]

rads = []
check_ns = []

for max_rad, min_rad, n_circles, check_n in zip(max_rads, min_rads, circles_per_wave, check_ns_per_wave):
    rads.append(np.logspace(np.log10(max_rad), np.log10(min_rad), int(n_circles)))
    check_ns.append(np.repeat(check_n, n_circles))


target_points = []
for tp, n_circles_per_wave in zip(tps, circles_per_wave):
    target_points += [tp] * n_circles_per_wave

rads = np.concatenate(rads)
check_ns = np.concatenate(check_ns)
init_rad = rads[0]

In [152]:
## initialize
start_circle = drawbox.centroid.buffer(init_rad)
main_orb = TargetPointGrowth(start_circle)

In [153]:
for i, rad in enumerate(tqdm(rads)):
    n_checks = check_ns[i]
    target_point = target_points[i]
    try:
        main_orb.grow_circle_diff(target_point=target_point, rad=rad, n_checks=n_checks)
    except:
        pass


100%|██████████| 100/100 [00:01<00:00, 91.99it/s]


In [154]:
main_orb_triangles = so.triangulate(MultiPoint([l.centroid for l in main_orb.lines]), edges=False)

In [78]:
# main_orb_triangles = MultiPolygon([t.buffer((t.bounds[1] - t.bounds[0]) * -0.000 - 0.00 ) for t in main_orb_triangles if t.within(main_orb.growth)])

In [89]:
# # function dis
# main_orb_hatches = []
# for t in main_orb_triangles:
#     h = hatch.hatchbox(t, (t.bounds[1] - t.bounds[0]) * 90, 0.02)
#     if h.length > 0:
#         if h.type == 'MultiLineString':
#             h = merge_MultiLineStrings(hatch.connect_hatchlines(h, 0.025))
#         main_orb_hatches.append(h)
# main_orb_hatches = merge_MultiLineStrings(main_orb_hatches)

In [176]:
tris = main_orb_triangles

In [177]:

ids =[id(tri) for tri in tris]
df = geopandas.GeoDataFrame({
    'geometry': tris,
    'id': ids,
    'distances': [[None]] * len(tris),
    'generation': [0] * len(tris),
    'subdivided': [False] * len(tris),
    'parent': [None] * len(tris),
    },
)
max_generation=7

In [178]:
distances = [0.3, 0.5, 0.3]

In [179]:
n_subdivisions = 340

In [180]:

for i in tqdm(range(n_subdivisions)):
    try:
        valid = (df['generation'] < max_generation) & ~df['subdivided']
        df = choose_among_valid_and_subdivide_recursive(df, valid, distances, n_iters=np.random.randint(1,4), n_per_iter=1)
    except:
        pass

100%|██████████| 340/340 [00:30<00:00, 10.97it/s]


In [149]:
layers = []
for gen, group in df.groupby('generation'):
    layer = gp.merge_Polygons([ls for ls in group.geometry if ls is not None]).buffer(-0.2, cap_style=2, join_style=2).boundary
    layers.append(layer)

In [181]:
layers = []
for gen, group in df.groupby('generation'):
    layer = gp.merge_LineStrings([ls for ls in group.boundary if ls is not None])
    layers.append(layer)

In [182]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# tolerance=0.5
# sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-08T01:18:22.261263

In [147]:
sk.vpype(f'splitall linemerge --tolerance 0.05mm linemerge --tolerance 0.05mm linemerge --tolerance 0.05mm linesort')

In [148]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/166_triangle_subdivision.svg')